In [11]:
import numpy as np
import pandas as pd
#from representatives import kMeans # you can use this kMeans in Ex. 3

# Exercise 1

In [69]:
def agglomerativeClustering(D, dist, k=1):
    C = [[i] for i in range(len(D))]

    while len(C) > k:
        distances = np.zeros((len(C), len(C)))
        for i in range(len(C)):
            for j in range(i + 1, len(C)):
                distances[i][j] = dist(D, C[i], C[j])
                distances[j][i] = distances[i][j]

        min_dist = np.inf
        min_i, min_j = None, None

        for i in range(len(C)):
            for j in range(i + 1, len(C)):
                if distances[i][j] < min_dist:
                    min_dist = distances[i][j]
                    min_i, min_j = i, j

        Ci, Cj = C[min_i], C[min_j]
        Cij = []
        Cij.extend(Ci)
        Cij.extend(Cj)

        C.remove(Ci)
        C.remove(Cj)
        C.append(Cij)

    C.sort(key=sortFunction)
    cluster_ids = [-1] * len(D)
    for i, cluster in enumerate(C):
        for point in cluster:
            cluster_ids[point] = i

    return cluster_ids

def sortFunction(elem):
    return min(elem)

In [68]:
def singleLink(D, Ci, Cj):
    min_dist = float()
    for i in Ci:
        for j in Cj:
            dist_ij = np.linalg.norm(D[i] - D[j])
            if dist_ij < min_dist:
                min_dist = dist_ij
    return min_dist

In [67]:
def completeLink(D, Ci, Cj):
    max_dist = 0
    for i in Ci:
        for j in Cj:
            dist_ij = np.linalg.norm(D[i] - D[j])
            if dist_ij > max_dist:
                max_dist = dist_ij
    return max_dist

In [66]:
def groupAverage(D, Ci, Cj):
    distance_sum = 0

    D_i = [D[i] for i in Ci]
    D_j = [D[i] for i in Cj]

    for point_i in D_i:
        for point_j in D_j:
            distance_sum += np.linalg.norm(point_i - point_j)

    return distance_sum / (len(D_i) * len(D_j))

In [65]:
def meanDistance(D, Ci, Cj):
    D_i = [D[i] for i in Ci]
    D_j = [D[i] for i in Cj]

    miu_i = np.average(D_i, axis=0)
    miu_j = np.average(D_j, axis=0)

    return np.linalg.norm(miu_j - miu_i)

In [64]:
def ward(D, Ci, Cj):
    D_i = [D[i] for i in Ci]
    D_j = [D[i] for i in Cj]

    miu_i = np.average(D_i, axis=0)
    miu_j = np.average(D_j, axis=0)

    mean_distance = np.linalg.norm(miu_j - miu_i)

    n_i = len(D_i)
    n_j = len(D_j)

    return (((n_i * n_j) / (n_i + n_j)) * (mean_distance**2))

In [21]:
def agglomerativeClusteringLW(D, dist, k=1):
    C = [[i] for i in range(len(D))]

    distances = np.zeros((len(C), len(C)))
    for i in range(len(C)):
        for j in range(i + 1, len(C)):
            distances[i][j] = np.linalg.norm(D[i] - D[j])
            distances[j][i] = distances[i][j]


    while len(C) > k:              
        min_dist = np.inf
        min_i, min_j = None, None

        for i in range(len(C)):
            for j in range(i + 1, len(C)):
                if distances[i][j] < min_dist:
                    min_dist = distances[i][j]
                    min_i, min_j = i, j

        Ci, Cj = C[min_i], C[min_j]
        Cij = []
        Cij.extend(Ci)
        Cij.extend(Cj)

        C.remove(Ci)
        C.remove(Cj)
        C.append(Cij)

        n_i = len(Ci)
        n_j = len(Cj)
        n_r = 0

        new_column = []
        for r in range(C-1):
            n_r = len(C[r])
            a_i, a_j, b, gamma = lanceWilliamsParams(dist, n_i, n_j, n_r)
            
            new_distance = a_i * distances[min_i][r] + a_j * distances[min_j][r] + b * distances[min_i][min_j] + gamma * np.abs(distances[min_i][r] - distances[min_j][r])
            new_column.append(new_distance)

        distances = np.delete(distances, [min_i,min_j], 0)
        distances = np.delete(distances, [min_i,min_j], 1)

        distances = np.insert(distances, len(distances), new_column, 0)
        new_column.append(0)
        distances = np.insert(distances, (len(distances)-1), new_column, 1)


    C.sort(key=sortFunction)
    cluster_ids = [-1] * len(D)
    for i, cluster in enumerate(C):
        for point in cluster:
            cluster_ids[point] = i

    return cluster_ids

def sortFunction(elem):
    return min(elem)

def lanceWilliamsParams(dist, n_i, n_j, n_r):
    if dist == 'single':
        return (1/2, 1/2, 0, -1/2)
    elif dist == 'complete':
        return (1/2, 1/2, 0, 1/2)
    elif dist == 'groupavg':
        return ((n_i/(n_i+n_j)), (n_j/(n_i+n_j)), 0, 0)
    elif dist == 'meandist':
        return ((n_i/(n_i+n_j)), (n_j/(n_i+n_j)), (-(n_i*n_j)/((n_i+n_j)**2)), 0)
    elif dist == 'ward':
        return (((n_i+n_r)/(n_i+n_j+n_r)), ((n_j+n_r)/(n_i+n_j+n_r)), (-n_r/(n_i+n_j+n_r)), 0)

In [89]:
A = np.arange(25).reshape(5,5)

print(A)
A = np.delete(A, [1,3], 0)
A = np.delete(A, [1,3], 1)
print(A)

col = [1,2,3]
A = np.insert(A, len(A), col,0)
print(A)

col.append(4)
print(col)
A = np.insert(A, (len(A)-1), col,1)
print(A)



[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[[ 0  2  4]
 [10 12 14]
 [20 22 24]]
[[ 0  2  4]
 [10 12 14]
 [20 22 24]
 [ 1  2  3]]
[1, 2, 3, 4]
[[ 0  2  4  1]
 [10 12 14  2]
 [20 22 24  3]
 [ 1  2  3  4]]


In [70]:
from sklearn.decomposition import PCA

'''
  Tries to make clustering c2 equal to clustering c1 by renaming the cluster names.
  If the clusterings are effectively equivalent, the output will be equal to c1.
'''
def try_unification(c1, c2):
    v1 = list(np.unique(c1))
    v2 = list(np.unique(c2))
    new_vals = []
    if len(c1) != len(c2):
        print("Cannot unify clusterings of different lengths!")
        return None
    if len(v1) != len(v2):
        print("Cannot unify clusterings of different numbers of clusters!")
        return None
    
    
    # use different symbols for clusterings
    i = 0
    for v in v2:
        c = "v" + str(i)
        while c in v1:
            i +=1
            c = "v" + str(i)
        new_vals.append(c)
        i += 1
    c_new = [new_vals[v2.index(i)] for i in c2]
    
    # replace occurrences
    targets = []
    for symbol in new_vals:
        first_index = c_new.index(symbol)
        replace_symbol = None
        for v in c1:
            if not v in targets:
                replace_symbol = v
                break
        if replace_symbol in targets:
            print("Warning: No unification possible, the symbol " + replace_symbol + " has already been addressed before!")
            return None
        
        c_new = [replace_symbol if v == symbol else v for v in c_new]
        targets.append(replace_symbol)
    return c_new

# Test Complete Link on Iris (PCA)
dfIris = pd.read_csv("iris.csv")
DIris = PCA(n_components=2).fit_transform(dfIris.values[:,:4])
labelsIris = list(pd.unique(dfIris["species"]))
C_perfect = np.array([labelsIris.index(l) for l in dfIris["species"]])
C_iris = np.array(try_unification(C_perfect, agglomerativeClustering(DIris, completeLink, k=3)))
M_expected = np.array([[50, 0, 0], [0, 14, 49], [0, 36, 1]]) # according to slide 13
for i in range(3):
    cond1 = C_iris == i
    for j in range(3):
        cond2 = C_perfect == j
        cnt_combo = np.count_nonzero(cond1 & cond2)
        print(labelsIris[i] + "/" + labelsIris[j] + ": " + ("OK" if cnt_combo == M_expected[i,j] else "FAILED. Expected " + str(M_expected[i,j]) + " but saw " + str(cnt_combo)))

# Test Coincidence of Standard and Lance-Williams
for D in [DIris, pd.get_dummies(pd.read_csv("Mall_Customers.csv")).values]:
    for pair in [("single", singleLink), ("complete", completeLink), ("groupavg", groupAverage), ("ward", ward)]:
        C_ac = np.array(agglomerativeClustering(D, pair[1], k=3))
        C_aclw = np.array(try_unification(C_ac, agglomerativeClusteringLW(D, pair[0], k=3)))
        mismatches = C_ac != C_aclw
        print(pair[0] + (" OK" if np.count_nonzero(mismatches) == 0 else (" FAILED. Difference in positions: " + str(np.where(mismatches)[0]) + ": " + str(C_ac[mismatches]) + " vs " + str(C_aclw[mismatches]))))

setosa/setosa: OK
setosa/versicolor: OK
setosa/virginica: OK
versicolor/setosa: OK
versicolor/versicolor: OK
versicolor/virginica: OK
virginica/setosa: OK
virginica/versicolor: OK
virginica/virginica: OK


NameError: name 'agglomerativeClusteringLW' is not defined

# Exercise 2

In [ ]:
df_mall = pd.read_csv("Mall_Customers.csv")

#k=5
#Run Aglomerative Clustering with Single Link

#Run Aglomerative Clustering with Group Average

#Run Aglomerative Clustering LW with Single Link

#Run Aglomerative Clustering LW with Group Average

In [ ]:
def getBestProjection(A, C, targetdims=2):

In [ ]:
#Plot 6x2 for every dataset